# C. PERFORMANCE UNDER NEW CATEGORIES
In this experiment, we will evaluate effect of the proposed few-shot learning method to address the fourth challenge in limited data fault diagnosis: 
especially in real-world applications, fault categories and working conditions  are usually unbalanced. It is thus difficult to collect enough samples for every fault type under different working conditions. We mainly pay attention to unbalanced fault categories. When new categories appear, traditional deep learning methods need to retrain to deal with the new categories diagnosis. However, different from the traditional deep learning methods,  the few-shot learning model can be directly used in the new  category diagnosis by just giving a few new category samples. We train the WDCNN model from all categories and the few-shot model from  30\% randomly new categories to all see categories. We repeated each such experiment ten times to deal with the randomness of the algorithms.  

# C. 新类别下的表现
在本实验中，我们将评估所提出的小样本学习方法在解决有限数据故障诊断中的第四个挑战的效果：
尤其是在实际应用中，故障类别和工作条件通常是不平衡的。 因此很难在不同的工作条件下为每种故障类型收集足够的样本。 我们主要关注不平衡故障类别。 当出现新的类别时，传统的深度学习方法需要重新训练来处理新的类别诊断。 然而，与传统的深度学习方法不同，few-shot学习模型只需给出几个新的类别样本即可直接用于新的类别诊断。 我们从所有类别训练 WDCNN 模型和从 30% 随机新类别到所有可见类别的少样本模型。 我们将每个这样的实验重复十次以处理算法的随机性。

In [1]:
%matplotlib inline
import tensorflow as tf
import tensorflow
# set the memory usage
from tensorflow.python.keras.backend import set_session

tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=tf_config))

import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
import imp
import pandas as pd
import keras
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping


## load data

In [2]:
import cwru 

window_size = 2048
data = cwru.CWRU(['12DriveEndFault'], ['1772', '1750', '1730'], window_size)
data.nclasses,data.classes,len(data.X_train),len(data.X_test)

Datasets/CWRU 12DriveEndFault 1730
idx： 0
fpath： Datasets/CWRU\NormalBaseline\1730\Normal.mat
time_series.shape: (485643, 2)
idx： 1
fpath： Datasets/CWRU\NormalBaseline\1750\Normal.mat
dict_keys(['__header__', '__version__', '__globals__', 'ans', 'X098_DE_time', 'X098_FE_time', 'X099_DE_time', 'X099_FE_time'])
time_series.shape: (483903, 2)
idx： 2
fpath： Datasets/CWRU\NormalBaseline\1772\Normal.mat
time_series.shape: (483903, 2)
idx： 3
fpath： Datasets/CWRU\12DriveEndFault\1730\0.007-Ball.mat
time_series.shape: (121556, 2)
idx： 4
fpath： Datasets/CWRU\12DriveEndFault\1750\0.007-Ball.mat
time_series.shape: (121556, 2)
idx： 5
fpath： Datasets/CWRU\12DriveEndFault\1772\0.007-Ball.mat
time_series.shape: (121410, 2)
idx： 6
fpath： Datasets/CWRU\12DriveEndFault\1730\0.014-Ball.mat
time_series.shape: (122136, 2)
idx： 7
fpath： Datasets/CWRU\12DriveEndFault\1750\0.014-Ball.mat
time_series.shape: (121991, 2)
idx： 8
fpath： Datasets/CWRU\12DriveEndFault\1772\0.014-Ball.mat
time_series.shape: (122136, 2

(10,
 [('NormalBaselineNormal', 0),
  ('12DriveEndFault0.007-Ball', 1),
  ('12DriveEndFault0.014-Ball', 2),
  ('12DriveEndFault0.021-Ball', 3),
  ('12DriveEndFault0.007-InnerRace', 4),
  ('12DriveEndFault0.014-InnerRace', 5),
  ('12DriveEndFault0.021-InnerRace', 6),
  ('12DriveEndFault0.007-OuterRace6', 7),
  ('12DriveEndFault0.014-OuterRace6', 8),
  ('12DriveEndFault0.021-OuterRace6', 9)],
 19800,
 750)

## Load models

In [3]:
import models
# imp.reload(models)
siamese_net = models.load_siamese_net_my_mew((window_size,2))
print('\nsiamese_net summary:')
siamese_net.summary()


siamese_net summary:
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 2048, 2)]    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 2048, 2)]    0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 100)          137763      input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           model

In [4]:
wdcnn_net = models.load_wdcnn_net()
print('\nwdcnn_net summary:')
wdcnn_net.summary()

52806

wdcnn_net summary:
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 2048, 2)]         0         
_________________________________________________________________
sequential (Sequential)      (None, 100)               51796     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 52,806
Trainable params: 52,806
Non-trainable params: 0
_________________________________________________________________


## Config

In [5]:
import keras
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping

import siamese
# imp.reload(siamese)
import utils
# imp.reload(utils)

settings = {
  "N_way": 10,           # how many classes for testing one-shot tasks>
  "batch_size": 32,
  "best": -1,
  "evaluate_every": 600,   # interval for evaluating on one-shot tasks
  "loss_every": 20,      # interval for printing loss (iterations)
  "n_iter": 15000,
  "n_val": 2,          #how many one-shot tasks to validate on?
  "n": 0,
  "save_path":"",
  "save_weights_file": "weights-best-10-oneshot-low-data.hdf5"
}


exp_name = "EXP-C"
exps = [90,1500]
see_rates = [0.7,0.8,0.9]
times = 10
# times = 2

## Training & Testing

In [6]:
is_training = True    # enable or disable train models. if enable training, save best models will be update.

train_classes = sorted(list(set(data.y_train)))
train_indices = [np.where(data.y_train == i)[0] for i in train_classes]
for exp in exps:
    # enable the random seed for experimental reproduction
    np.random.seed(exp)
    scores_1_shot = []
    scores_5_shot = []
    num = int(exp/len(train_classes))
    settings['evaluate_every'] = 300 if exp<1000 else 600
    print(settings['evaluate_every'])
    for time_idx in range(times):
        np.random.seed(0)
        train_idxs = []
        val_idxs = []
        for i, c in enumerate(train_classes):
            select_idx = train_indices[i][np.random.choice(len(train_indices[i]), num, replace=False)]
            split = int(0.6*num)
            train_idxs.extend(select_idx[:split])
            end = num
            if(0.4*num>100):
                end = split+100
            val_idxs.extend(select_idx[split:end])

        X_train, y_train = data.X_train[train_idxs],data.y_train[train_idxs] 
        X_val, y_val = data.X_train[val_idxs],data.y_train[val_idxs] 

        print(train_idxs[0:10])
        print(val_idxs[0:10])   
            
        random_train_classes = [0]+list(np.random.choice(list(range(1,10)), 9, replace=False))
        for rate in see_rates:
            print("\n%s-%s-%s"%(exp,rate,time_idx) + '*'*80)
            settings["save_path"] = "tmp/%s/size_%s/%s/time_%s/" % (exp_name,exp,rate,time_idx)
            data._mkdir(settings["save_path"])
            random_select_classes = random_train_classes[:int(rate*len(train_classes))]
            print(random_select_classes)
            train_idxs_rate = np.array([np.where(y_train == i)[0] for i in random_select_classes]).reshape(-1)
            val_idxs_rate =  np.array([np.where(y_val == i)[0] for i in random_select_classes]).reshape(-1)
            X_train_rate, y_train_rate = X_train[train_idxs_rate],y_train[train_idxs_rate]
            X_val_rate, y_val_rate = X_val[val_idxs_rate],y_val[val_idxs_rate]

            # load one-shot model and training
            siamese_net = models.load_siamese_net_my_mew()
            siamese_loader = siamese.Siamese_Loader(X_train_rate,
                                            y_train_rate,
                                            X_val_rate,
                                            y_val_rate)

            if(is_training):
                print(siamese.train_and_test_oneshot(settings,siamese_net,siamese_loader))
                
            print("load best weights",settings["save_path"] + settings['save_weights_file'])
            siamese_net.load_weights(settings["save_path"] + settings['save_weights_file'])


            siamese_loader = siamese.Siamese_Loader(X_train,
                                            y_train,
                                            data.X_test,
                                            data.y_test)
            s = 'val'
            preds_5_shot = []
            scores = []
            for k in range(5):
                val_acc, preds, prods = siamese_loader.test_oneshot2(siamese_net,len(siamese_loader.classes[s]),
                                                             len(siamese_loader.data[s]),verbose=False)
    #             confusion_plot(preds[:,1],preds[:,0])
                print('测试集的正确率:',val_acc)
                scores.append(val_acc)
                preds_5_shot.append(preds[:,1])
            preds = []
            for line in np.array(preds_5_shot).T:
                preds.append(np.argmax(np.bincount(line)))
    #         confusion_plot(np.array(preds),data.y_test) 
            score_5_shot = accuracy_score(data.y_test,np.array(preds))*100
            print('5_shot:',score_5_shot)
            scores_1_shot.append(scores[0])
            scores_5_shot.append(score_5_shot)

    a =pd.DataFrame(np.array(scores_1_shot).reshape(-1,len(see_rates)))
    a.columns = see_rates
    a.to_csv("tmp/%s/size_%s/scores_1_shot.csv" % (exp_name,exp),index=True)


    a =pd.DataFrame(np.array(scores_5_shot).reshape(-1,len(see_rates)))
    a.columns = see_rates
    a.to_csv("tmp/%s/size_%s/scores_5_shot.csv" % (exp_name,exp),index=True)

300
[2281, 1786, 1535, 3938, 6914, 8697, 1920, 3126, 7442, 6055]
[9884, 4896, 3931, 265, 19713, 12543, 14017, 6065, 16988, 5480]

90-0.7-0********************************************************************************
[0, 5, 7, 8, 1, 6, 4]
Siamese_Loader的X_train的shape (35, 2048, 2)
Siamese_Loader的y_train的shape (35,)
Siamese_Loader的X_val的shape (28, 2048, 2)
Siamese_Loader的y_val的shape (28,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 300, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-C/size_90/0.7/time_0/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 89.28571428571429
580 轮: 损失值：0.00036, 正确率:1.00000，验证集正确率92.85714:
iteration 600 evaluating: 89.28571428571429
880 轮: 损失值：0.00030, 正确率:1.00000，验证集正确率89.28571:
iteration 900 evaluating: 92.85714285714286
1180 轮: 损失值：0.00009, 正确率:1.00000，验证集正确率96.42857::
iteration 1200 evaluating: 96.42857142857143
1480 轮: 损失值：0.00007, 正确率:1.00000，验证集正确率96.4

Siamese_Loader的X_train的shape (35, 2048, 2)
Siamese_Loader的y_train的shape (35,)
Siamese_Loader的X_val的shape (28, 2048, 2)
Siamese_Loader的y_val的shape (28,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 300, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-C/size_90/0.7/time_1/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 85.71428571428571
280 轮: 损失值：0.00066, 正确率:1.00000，验证集正确率92.85714:
iteration 300 evaluating: 92.85714285714286
880 轮: 损失值：0.00026, 正确率:1.00000，验证集正确率92.85714::
iteration 900 evaluating: 92.85714285714286
1480 轮: 损失值：0.00013, 正确率:1.00000，验证集正确率92.85714:
iteration 1500 evaluating: 92.85714285714286
1780 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率92.85714::
iteration 1800 evaluating: 92.85714285714286
2080 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率96.42857::
iteration 2100 evaluating: 96.42857142857143
2680 轮: 损失值：0.00053, 正确率:1.00000，验证集正确率100.00000:
iteration 2700 evaluating: 100.0
2980 轮: 损失值

14680 轮: 损失值：0.00001, 正确率:1.00000，验证集正确率100.00000:
iteration 14700 evaluating: 100.0
14980 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率100.00000:100.0
load best weights tmp/EXP-C/size_90/0.8/time_1/weights-best-10-oneshot-low-data.hdf5
Siamese_Loader的X_train的shape (50, 2048, 2)
Siamese_Loader的y_train的shape (50,)
Siamese_Loader的X_val的shape (750, 2048, 2)
Siamese_Loader的y_val的shape (750,)
测试集的正确率: 82.93333333333334
测试集的正确率: 83.06666666666666
测试集的正确率: 81.86666666666666
测试集的正确率: 82.4
测试集的正确率: 83.86666666666666
5_shot: 82.93333333333334

90-0.9-1********************************************************************************
[0, 5, 7, 8, 1, 6, 4, 2, 3]
Siamese_Loader的X_train的shape (45, 2048, 2)
Siamese_Loader的y_train的shape (45,)
Siamese_Loader的X_val的shape (36, 2048, 2)
Siamese_Loader的y_val的shape (36,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 300, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-C/size_90/0.9/time_1/', 'save_weights_file': 'weights-best

2080 轮: 损失值：0.00011, 正确率:1.00000，验证集正确率96.87500:
iteration 2100 evaluating: 96.875
3280 轮: 损失值：0.00218, 正确率:1.00000，验证集正确率100.00000:
iteration 3300 evaluating: 96.875
4780 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率96.87500::
iteration 4800 evaluating: 96.875
5680 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率96.87500::
iteration 5700 evaluating: 96.875
6280 轮: 损失值：0.00260, 正确率:1.00000，验证集正确率90.62500::
iteration 6300 evaluating: 96.875
6880 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率96.87500:
iteration 6900 evaluating: 96.875
7180 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率96.87500::
iteration 7200 evaluating: 100.0
9280 轮: 损失值：0.00001, 正确率:1.00000，验证集正确率84.37500::
iteration 9300 evaluating: 100.0
9880 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率100.00000:
iteration 9900 evaluating: 100.0
10480 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率100.00000:
iteration 10500 evaluating: 100.0
12280 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率100.00000:
iteration 12300 evaluating: 100.0
12580 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率100.00000:
iteration 12600 evaluating: 

3880 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率96.87500:
iteration 3900 evaluating: 96.875
4480 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率96.87500:
iteration 4500 evaluating: 96.875
4780 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率100.00000:
iteration 4800 evaluating: 100.0
5380 轮: 损失值：0.00428, 正确率:1.00000，验证集正确率100.00000:
iteration 5400 evaluating: 100.0
5680 轮: 损失值：0.00009, 正确率:1.00000，验证集正确率100.00000:
iteration 5700 evaluating: 100.0
6280 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率90.62500::
iteration 6300 evaluating: 100.0
6580 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率100.00000:
iteration 6600 evaluating: 100.0
7180 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率100.00000:
iteration 7200 evaluating: 100.0
7480 轮: 损失值：0.00001, 正确率:1.00000，验证集正确率100.00000:
iteration 7500 evaluating: 100.0
7780 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率100.00000:
iteration 7800 evaluating: 100.0
8080 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率100.00000:
iteration 8100 evaluating: 100.0
8380 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率100.00000:
iteration 8400 evaluating: 100.0
8680

...

测试集的正确率: 94.93333333333334
测试集的正确率: 95.86666666666666
测试集的正确率: 95.06666666666666
测试集的正确率: 94.13333333333334
测试集的正确率: 94.4
5_shot: 95.46666666666667
[2281, 1786, 1535, 3938, 6914, 9884, 4896, 3931, 265, 2843]
[18189, 957, 7915, 1694, 11220, 18944, 8580, 457, 2697, 15560]

1500-0.7-6********************************************************************************
[0, 5, 7, 8, 1, 6, 4]
Siamese_Loader的X_train的shape (630, 2048, 2)
Siamese_Loader的y_train的shape (630,)
Siamese_Loader的X_val的shape (420, 2048, 2)
Siamese_Loader的y_val的shape (420,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 600, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-C/size_1500/0.7/time_6/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 85.95238095238095
580 轮: 损失值：0.00303, 正确率:1.00000，验证集正确率100.00000:
iteration 600 evaluating: 100.0
1780 轮: 损失值：0.00041, 正确率:1.00000，验证集正确率99.76190::
iteration 1800 evaluating: 100.0
2380 轮: 

13780 轮: 损失值：0.00010, 正确率:1.00000，验证集正确率100.00000:
iteration 13800 evaluating: 100.0
14380 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率100.00000:
iteration 14400 evaluating: 100.0
14980 轮: 损失值：0.00007, 正确率:1.00000，验证集正确率100.00000:100.0
load best weights tmp/EXP-C/size_1500/0.9/time_6/weights-best-10-oneshot-low-data.hdf5
Siamese_Loader的X_train的shape (900, 2048, 2)
Siamese_Loader的y_train的shape (900,)
Siamese_Loader的X_val的shape (750, 2048, 2)
Siamese_Loader的y_val的shape (750,)
测试集的正确率: 90.66666666666667
测试集的正确率: 89.33333333333333
测试集的正确率: 89.06666666666666
测试集的正确率: 90.66666666666667
测试集的正确率: 89.86666666666666
5_shot: 90.66666666666666
[2281, 1786, 1535, 3938, 6914, 9884, 4896, 3931, 265, 2843]
[18189, 957, 7915, 1694, 11220, 18944, 8580, 457, 2697, 15560]

1500-0.7-7********************************************************************************
[0, 5, 7, 8, 1, 6, 4]
Siamese_Loader的X_train的shape (630, 2048, 2)
Siamese_Loader的y_train的shape (630,)
Siamese_Loader的X_val的shape (420, 2048, 2)
Siamese_Loa

8980 轮: 损失值：0.00012, 正确率:1.00000，验证集正确率100.00000:
iteration 9000 evaluating: 100.0
9580 轮: 损失值：0.00011, 正确率:1.00000，验证集正确率100.00000:
iteration 9600 evaluating: 100.0
10180 轮: 损失值：0.00010, 正确率:1.00000，验证集正确率100.00000:
iteration 10200 evaluating: 100.0
10780 轮: 损失值：0.00010, 正确率:1.00000，验证集正确率100.00000:
iteration 10800 evaluating: 100.0
11380 轮: 损失值：0.00051, 正确率:1.00000，验证集正确率100.00000:
iteration 11400 evaluating: 100.0
12580 轮: 损失值：0.02042, 正确率:0.96875，验证集正确率100.00000:
iteration 12600 evaluating: 100.0
13180 轮: 损失值：0.00018, 正确率:1.00000，验证集正确率100.00000:
iteration 13200 evaluating: 100.0
13780 轮: 损失值：0.00042, 正确率:1.00000，验证集正确率100.00000:
iteration 13800 evaluating: 100.0
14380 轮: 损失值：0.00015, 正确率:1.00000，验证集正确率100.00000:
iteration 14400 evaluating: 100.0
14980 轮: 损失值：0.00013, 正确率:1.00000，验证集正确率100.00000:100.0
load best weights tmp/EXP-C/size_1500/0.9/time_7/weights-best-10-oneshot-low-data.hdf5
Siamese_Loader的X_train的shape (900, 2048, 2)
Siamese_Loader的y_train的shape (900,)
Siamese_Loader的X

6580 轮: 损失值：0.00018, 正确率:1.00000，验证集正确率100.00000:
iteration 6600 evaluating: 100.0
7180 轮: 损失值：0.00018, 正确率:1.00000，验证集正确率100.00000:
iteration 7200 evaluating: 100.0
7780 轮: 损失值：0.00010, 正确率:1.00000，验证集正确率100.00000:
iteration 7800 evaluating: 100.0
8380 轮: 损失值：0.00034, 正确率:1.00000，验证集正确率100.00000:
iteration 8400 evaluating: 100.0
8980 轮: 损失值：0.00010, 正确率:1.00000，验证集正确率100.00000:
iteration 9000 evaluating: 100.0
10180 轮: 损失值：0.00006, 正确率:1.00000，验证集正确率100.00000:
iteration 10200 evaluating: 100.0
11380 轮: 损失值：0.00010, 正确率:1.00000，验证集正确率100.00000:
iteration 11400 evaluating: 100.0
11980 轮: 损失值：0.00011, 正确率:1.00000，验证集正确率100.00000:
iteration 12000 evaluating: 100.0
12580 轮: 损失值：0.00011, 正确率:1.00000，验证集正确率100.00000:
iteration 12600 evaluating: 100.0
13780 轮: 损失值：0.00006, 正确率:1.00000，验证集正确率100.00000:
iteration 13800 evaluating: 100.0
14380 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率100.00000:
iteration 14400 evaluating: 100.0
14980 轮: 损失值：0.00006, 正确率:1.00000，验证集正确率100.00000:100.0
load best weights t

5380 轮: 损失值：0.00020, 正确率:1.00000，验证集正确率100.00000:
iteration 5400 evaluating: 100.0
6580 轮: 损失值：0.00154, 正确率:1.00000，验证集正确率100.00000:
iteration 6600 evaluating: 100.0
7180 轮: 损失值：0.00017, 正确率:1.00000，验证集正确率100.00000:
iteration 7200 evaluating: 100.0
7780 轮: 损失值：0.00015, 正确率:1.00000，验证集正确率100.00000:
iteration 7800 evaluating: 100.0
8380 轮: 损失值：0.00016, 正确率:1.00000，验证集正确率100.00000:
iteration 8400 evaluating: 100.0
8980 轮: 损失值：0.00017, 正确率:1.00000，验证集正确率100.00000:
iteration 9000 evaluating: 100.0
9580 轮: 损失值：0.00018, 正确率:1.00000，验证集正确率100.00000:
iteration 9600 evaluating: 100.0
10180 轮: 损失值：0.00020, 正确率:1.00000，验证集正确率100.00000:
iteration 10200 evaluating: 100.0
10780 轮: 损失值：0.00015, 正确率:1.00000，验证集正确率100.00000:
iteration 10800 evaluating: 100.0
11380 轮: 损失值：0.00011, 正确率:1.00000，验证集正确率100.00000:
iteration 11400 evaluating: 100.0
11980 轮: 损失值：0.00007, 正确率:1.00000，验证集正确率100.00000:
iteration 12000 evaluating: 100.0
13180 轮: 损失值：0.00021, 正确率:1.00000，验证集正确率100.00000:
iteration 13200 evaluating: 

## Analysis

In [7]:
exps = [90,1500]

scores_1_shot_all = pd.DataFrame()
scores_5_shot_all = pd.DataFrame()

for exp in exps:
    file_path = "tmp/%s/size_%s" % (exp_name,exp)
    tmp_data = pd.read_csv("%s/scores_1_shot.csv" % (file_path), 
                           sep=',', index_col=0)
    tmp_data = pd.DataFrame(tmp_data.values.reshape(-1,len(see_rates)))
    tmp_data.columns = see_rates
    tmp_data['model'] = 'One-shot'
    tmp_data['exp'] = exp 
    scores_1_shot_all = pd.concat([scores_1_shot_all,tmp_data],axis=0)

    tmp_data = pd.read_csv("%s/scores_5_shot.csv" % (file_path), 
                           sep=',', index_col=0)
    tmp_data = pd.DataFrame(tmp_data.values.reshape(-1,len(see_rates)))
    tmp_data.columns = see_rates
    tmp_data['model'] = 'Five-shot'
    tmp_data['exp'] = exp 
    scores_5_shot_all = pd.concat([scores_5_shot_all,tmp_data],axis=0)


scores_1_shot_all.to_csv("tmp/%s/scores_1_shot_all.csv" % (exp_name), float_format='%.6f', index=True)
scores_5_shot_all.to_csv("tmp/%s/scores_5_shot_all.csv" % (exp_name), float_format='%.6f', index=True)

scores_1_shot_all['model'] = 'One-shot'
scores_5_shot_all['model'] = 'Five-shot'

scores_all = pd.concat([scores_1_shot_all,scores_5_shot_all],axis=0)
scores_all.to_csv("tmp/%s/scores_all.csv" % (exp_name), float_format='%.6f', index=True)   

scores_all

,0.7,0.8,0.9,model,exp
0,78.933333,87.066667,87.733333,One-shot,90
1,84.000000,82.933333,90.533333,One-shot,90
2,79.333333,83.866667,81.866667,One-shot,90
3,79.333333,88.400000,89.733333,One-shot,90
4,77.200000,83.600000,88.666667,One-shot,90
5,77.733333,85.600000,93.866667,One-shot,90
6,84.000000,81.333333,90.400000,One-shot,90
7,80.933333,91.333333,79.866667,One-shot,90
8,79.600000,81.866667,90.400000,One-shot,90
9,78.133333,86.133333,88.800000,One-shot,90


In [8]:
scores_all_mean = scores_all.groupby(['exp','model']).mean()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_std = scores_all.groupby(['exp','model']).std()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_mean.to_csv("tmp/%s/scores_all_mean.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_std.to_csv("tmp/%s/scores_all_std.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_mean, scores_all_std

(                      0.7        0.8        0.9
 exp  model                                     
 90   One-shot   79.920000  85.213333  88.186667
      Five-shot  80.053333  85.013333  88.093333
 1500 One-shot   79.120000  87.346667  94.186667
      Five-shot  78.840000  87.986667  94.360000,
                      0.7       0.8       0.9
 exp  model                                  
 90   One-shot   2.391095  3.125333  4.215527
      Five-shot  2.147361  3.025619  4.452903
 1500 One-shot   2.624255  2.253513  2.248906
      Five-shot  2.448925  2.028391  2.296578)